Using kernel `conda_pytorch_latest_p36`

In [1]:
# !pip install transformers

In [2]:
import sys
sys.path.append('../../../')

In [3]:
from deep.constants import *

In [4]:
import torch

In [5]:
from pathlib import Path
import os
import random

In [6]:
import pandas as pd
import numpy as np
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import classification_report

In [7]:
data = IMMAP_PATH

In [8]:
train = pd.read_csv(data / 'final_data/en' / 'sentences_en_train.csv')
test = pd.read_csv(data / 'final_data/en' / 'sentences_en_test.csv')

In [9]:
def process_for_sector(df, sector, train):
    relevant_train = df[df.is_relevant == 1]
    relevant_train.sector_ids = relevant_train.sector_ids.apply(eval)
    relevant_train = relevant_train[relevant_train.sector_ids.apply(len) > 0]
    
    positive_train = relevant_train[relevant_train.sector_ids.apply(lambda x: sector in x)]
    negative_train = relevant_train[relevant_train.sector_ids.apply(lambda x: sector not in x)]

    positive_train.sector_ids = 1
    negative_train.sector_ids = 0
    train_df = pd.concat([positive_train, negative_train])
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    
    train_df['is_valid'] = False if train else True
        
    return train_df

In [10]:
train_df = process_for_sector(train, 4, True)
test_df = process_for_sector(test, 4, False)

/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

In [12]:
train_df

,doc_id,sentence_id,sentence_text,is_relevant,sector_ids,is_valid
0,44635,16,"Khaled karzeh, an SCD volunteer at the center ...",1,1,False
1,45354,5,"As displaced continues to grow in the country,...",1,0,False
2,37691,0,"Ar-Raqqa - Wednesday, August 19th 2020* *- Tod...",1,1,False
3,40332,4,Some of the worst floods in decades followed a...,1,0,False
4,34594,227,"Moreover, 38 collective shelters have latrines...",1,0,False
...,...,...,...,...,...,...
22967,40930,20,The ward has opened initially for moderate COV...,1,1,False
22968,50728,3,There has been a drop in the number of cases r...,1,1,False
22969,49804,126,The national average price also in- creased by...,1,0,False
22970,50966,4,"A risk of Famine (IPC Phase 5) persists, and F...",1,0,False


In [13]:
train_encodings = tokenizer(list(train_df.sentence_text), truncation=True, padding=True)
train_labels = list(train_df.sector_ids)

In [14]:
test_encodings = tokenizer(list(test_df.sentence_text), truncation=True, padding=True)
test_labels = list(test_df.sector_ids)

In [15]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)

In [16]:
# compute metrics function for binary classification
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return 

In [17]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total # of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
    logging_steps=1,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=train_dataset            # evaluation dataset
)

In [18]:
len(train_dataset)

22972

In [19]:
torch.cuda.is_available()

False

In [ ]:
trainer.train()

Step,Training Loss
1,0.749200
2,0.806200
3,0.735300
4,0.667200
5,0.643600
6,0.584000
7,0.748200
8,0.726100
9,0.781300
10,0.657300


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-3435b262f1ae>", line 1, in <module>
    trainer.train()
  File "/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/transformers/trainer.py", line 1272, in train
    tr_loss += self.training_step(model, inputs)
  File "/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/transformers/trainer.py", line 1752, in training_step
    loss.backward()
  File "/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/torch/tensor.py", line 245, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/torch/autograd/__init__.py", line 145, in backward
    Variable._execution_engine.run_backward(


KeyboardInterrupt


  File "/Users/stefano/miniconda3/envs/deep-exp/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
outputs = trainer.predict(test_dataset)

In [ ]:
preds = np.argmax(outputs.predictions, axis=1)
labels = outputs.label_ids

In [ ]:
rep = classification_report(labels, preds)

In [ ]:
print(rep)